## Experiences
- méthode K-fold: La validation croisée (Cross-Validation) k-fold signifie que l’ensemble de données se divise en un nombre K. Elle divise l’ensemble de données au point où l’ensemble de test utilise chaque pli. Comprenons le concept à l’aide de la validation croisée à 5 volets ou K+5. Dans ce scénario, la méthode divise l’ensemble de données en cinq volets. Le modèle utilise le premier pli dans la première itération pour tester le modèle. Il utilise les autres ensembles de données pour former le modèle. Le deuxième pli aide à tester l’ensemble de données et les autres soutiennent le processus de formation. Le même processus se répète jusqu’à ce que l’ensemble de test utilise chaque pli des cinq plis.    
==> elle résulte généralement sur un modèle moins biaisé. Pour cause, elle permet d'assurer que toutes les observations de l'ensemble de données original aient la chance d'apparaître dans l'ensemble d'entraînement et dans l'ensemble de test.
- matrice de confusion : La matrice de confusion est un outil de mesure de la performance des modèles de classification à 2 classes ou plus. Dans le cas binaire (i.e. à deux classes, le cas le plus simple), la matrice de confusion est un tableau à 4 valeurs représentant les différentes combinaisons de valeurs réelles et valeurs prédites. Cette matrice est indispensable pour définir les différentes métriques de classification telles que l’Accuracy, le F1-score.
- mesures de performances :
  - recall = VP/VP+FN /si =1 alors tous les spams ont été détectés
  - precision = VP+FP /si =1 alors tous les spams détectés sont réellement des spams
- overfitting = sur-apprentissage : un modèle trop spécialisé sur les données du Training Set et qui se généralisera mal
- Underfitting = sous-apprentissage : un modèle généraliste incapable de fournir des prédictions précises    
==> on cherche un modèle qui est au “juste milieu”. EXEMPLE sur l’algorithme de Gradient Descent. Pour rappel l’algorithme de Gradient Descent permet de trouver le meilleur modèle prédictif en minimisant le coût d’erreur global de la phase d’apprentissage. Quand l’algorithme de Machine Learning apprend depuis le Training Set, les paramètres du modèle prédictif vont s’affiner au fil du temps. En conséquence, le coût d’erreur sur le Training Set continuera à se réduire. Parallèlement, le coût d’erreur des prédictions sur les données de test (non vu par l’algorithme) se réduira également. Mais si on continue à entrainer longuement le modèle, le coût d’erreur sur le Training Set continuera à se réduire. MAIS celui du Test Set pourra commencer à augmenter. Cette augmentation signifie que le modèle prédictif s’est trop spécialisé sur les données d’apprentissage. Le point où commence l’augmentation du coût d’erreur dans le Test Set est le commencement du Overfitting. Le juste milieu est le point juste avant le commencement de l’augmentation du coût d’erreur dans le Test Set. C’est à ce stade que le modèle sera abouti et bien généralisable.

## SVM

In [10]:
import pandas as pd
import numpy as np
import re
import nltk
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from matplotlib import pyplot
from sklearn.metrics import precision_recall_fscore_support as score 
from sklearn.model_selection import train_test_split
from sklearn import svm


In [7]:
# resumé de l'étape : cleaning
data = pd.read_csv('./Data/SMSSpamCollection.txt', sep='\t', header=None)
data.columns = ['label', 'content']

en_stop_words = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()


def clean_data(texte):
    texte_punct = "".join([ch for ch in texte if ch not in string.punctuation])
    tokens = re.split("\W+", texte_punct)
    texte_clean = [ps.stem(word)
                   for word in tokens if word not in en_stop_words]
    return texte_clean




# resumé de l'étape : vectorisation & Feature Engineering
def count_punct(texte):
    binary_punct = [1 for ch in texte if ch in string.punctuation]
    nb_punct = sum(binary_punct)
    totale = len(texte) - texte.count(" ")
    return round(nb_punct/totale,4)*100, totale

data['punct_rate'] = data['content'].apply(lambda x: count_punct(x)[0])
data['content_len'] = data['content'].apply(lambda x: count_punct(x)[1])

vectorization_full = CountVectorizer(analyzer=clean_data)
vect_final = vectorization_full.fit_transform(data['content'])
pd_vect_final = pd.DataFrame(vect_final.toarray())
pd_vect_final.columns = vectorization_full.get_feature_names_out()

all_data = pd.concat([pd_vect_final,
                     data['punct_rate'], data['content_len']], axis = 1)

all_data

,,0,008704050406,0089mi,0121,01223585236,01223585334,0125698789,02,020603,...,zogtoriu,zoom,zouk,zyada,é,ü,üll,〨ud,punct_rate,content_len
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,9.78,92
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25.00,24
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4.69,128
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,15.38,39
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4.08,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6.11,131
5568,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,3.45,29
5569,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,14.58,48
5570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.00,100


In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(all_data, data['label'], test_size= 0.2)

In [11]:
algo_svm = svm.SVC(kernel='linear')
algo_svm.fit(X_train, Y_train)

SVC(kernel='linear')